In [1]:
import numpy as np
from linear import LinintGrow, LinintEqui
from DiscretizeTools import grow_grid, rouwenhorst
from scipy.optimize import root



In [2]:
T = 40
J = 12
JR = 10
NP = 2
NS = 5
NA = 100

γ = 0.5
egam = 1 - 1/γ
ν = 0.335
β = 0.998**5

σθ = 0.23
σϵ = 0.05
ρ = 0.98

α = 0.36
δ = 1 - (1 - 0.0823)**5
Ω = 1.60

al = 0
au = 35
agrow = 0.05

n_p = (1 + 0.01)**5 -1
itermax = 50

damp = 0.3
sig = 1e-4



In [3]:
r = np.zeros(T+1)
rn = np.zeros(T+1)
w = np.zeros(T+1)
wn = np.zeros(T+1)
pp = np.zeros(T+1)

In [4]:
pop = np.zeros((J, T+1))
m = np.zeros((J, T+1))
aplus = np.zeros((J, NA + 1, NP, NS, T + 1))
c = np.zeros((J, NA + 1, NP, NS, T + 1))
l = np.zeros((J, NA + 1, NP, NS, T + 1))
VV = np.zeros((J, NA + 1, NP, NS, T + 1))
v = np.zeros((J, NA + 1, NP, NS, T + 1))
RHS = np.zeros((J, NA + 1, NP, NS, T + 1))
EV = np.zeros((J, NA + 1, NP, NS, T + 1))
Φ = np.zeros((J, NA + 1, NP, NS, T + 1))


c_coh = np.zeros((J,T+1))
l_coh = np.zeros((J,T+1))
y_coh = np.zeros((J,T+1))
a_coh = np.zeros((J,T+1))
v_coh = np.zeros((J,T+1))
VV_coh = np.zeros((J,T+1))
FLC = np.zeros((J,T+1))
m = np.zeros((J,T+1))
PP = np.zeros(T+1)

taxrev = np.zeros((4,T+1))

eff = np.zeros(J)

distθ = [0.5,0.5]

θ = np.exp(np.array([-σθ**0.5,σθ**0.5]))

η, π = rouwenhorst(ρ, σϵ, NS)
η = np.exp(η)


for j in range(J):
    pop[j, 0] = 1/(1+n_p)**j
    
for j in range(J):
    m[j, 0] = pop[j,0]/pop[0,0]

a = grow_grid(al, au, agrow, n=NA+1)


for j in range(J):
    for p in range(NP):
        for s in range(NS):
            aplus[j, :, p, s, 0] = np.maximum(a/2, np.ones(NA+1)*a[0]/2)

eff[0] = 1.0000
eff[1] = 1.3527
eff[2] = 1.6952
eff[3] = 1.8279
eff[4] = 1.9606
eff[5] = 1.9692
eff[6] = 1.9692
eff[7] = 1.9392
eff[8] = 1.9007
eff[JR-1:] = 0 

tax = np.ones(T+1)*2
τc = np.ones(T+1)*0.075
τw = np.zeros(T+1)
τr = np.zeros(T+1)
τp = np.ones(T+1)*0.1
κ = np.ones(T+1)*0.5

gy = 0.19
by = 0.6/5

K = np.ones(T+1) 
L = np.ones(T+1)
Y = np.ones(T+1)
C = np.ones(T+1)
H = np.ones(T+1)
A = np.ones(T+1)
BA = np.ones(T+1)
I = (n_p+δ)*K
INC = np.ones(T+1)
DIFF = np.ones(T+1)

G = np.ones(T+1)*gy*Y[0]
B = np.ones(T+1)*by*Y[0]

pen = np.zeros((J,T+1))
pen[JR-1:,0] = κ[0]
    


In [5]:
def prices(t):    
    r[t] = Ω*α*(K[t]/L[t])**(α-1)-δ
    w[t] = Ω*(1-α)*(K[t]/L[t])**α
    rn[t] = r[t]*(1-τr[t])
    wn[t] = w[t]*(1-τw[t]-τp[t])
    pp[t] = 1+τc[t]
    return None


In [6]:
# current time is t, calculate at which year the agent with age j turns to be age p
def year(t,j,p):
    year = t + p -j
    
    if t == 0 or year <= 0 : year = 0
        
    if t == T or year >= T : year =  T
    
    return year

In [7]:
def valuefunc(ap, cons, lab, j, p, s, t):
    chelp = max(cons,1e-10)
    lhelp = min(max(lab,0),1-1e-10)
    
    tp = year(t,j,j+1)
    
    ial, iar, ϕ = LinintGrow(ap,al,au,agrow,NA+1)
    
    val = 0
    
    if j < J-1:
        val = max(ϕ*EV[j+1, ial, p, s, tp] + (1-ϕ)*EV[j+1, iar, p, s, tp], 1e-10)**(1-1/γ)/(1-1/γ)

    val = (chelp**ν*(1-lhelp)**(1-ν))**(1-1/γ)/(1-1/γ) + β*val
    return val
    
    
    
    

In [8]:
def margu(cons,lab,it):
    return ν*(cons**ν*(1-lab)**(1-ν))**(1-1/γ)/(pp[it]*cons)


def interpolate(j,t):
    for i in range(NA+1):
        for p in range(NP):
            for s in range(NS):
                RHS[j,i,p,s,t] = 0
                EV[j,i,p,s,t] = 0
                
                for sp in range(NS):
                    chelp = max(c[j,i,p,sp,t],1e-10)
                    lhelp = max(l[j,i,p,sp,t],1e-10)
                    RHS[j,i,p,s,t] += π[s,sp]*margu(chelp,lhelp,t)
                    EV[j,i,p,s,t]  += π[s,sp]*VV[j,i,p,sp,t]
                RHS[j,i,p,s,t] = ((1+rn[t])*β*RHS[j,i,p,s,t])**(-γ)
                EV[j,i,p,s,t] = ((1-1/γ)*EV[j,i,p,s,t])**(1/(1-1/γ))
    return None
    
    

In [9]:
def foc(xin,i, j, p, s, t):
    ap = xin
    tp = year(t,j,j+1)
    vind = v[j, i, p, s, t]
    
    wage = wn[t]*eff[j]*θ[p]*η[s]
    
    available = (1 + rn[t])*a[i] + pen[j,t] + vind
    
    if j < JR-1:
        labcom = min(max(ν + (1-ν)*(ap-available)/wage, 0),1-1e-10)
    else:
        labcom = 0
        
    conscom = max((available + wage*labcom -ap)/pp[t],1e-10) 
    
    ial,iar,φ = LinintGrow(ap, al, au, agrow, NA)
    
    tomorrow = max(φ*RHS[j+1,ial,p,s,tp]+(1-φ)*RHS[j+1, iar,p,s,tp],0)
    foc = margu(conscom, labcom, t)**(-γ) - tomorrow
    
    return foc
        
        

In [10]:
def solve_household(j, t):
    it = year(t, j, J-1)
    
    for i in range(NA+1):
        aplus[J-1,i,:,:,it] = 0
        c[J-1,i,:,:,it] = ((1+rn[it])*a[i]+ pen[J-1,it] + v[J-1,i,:,:,it])/pp[it]
        l[J-1,i,:,:,it] = 0
        VV[J-1,i,:,:,it] = valuefunc(0, c[J-1, i, 0, 0, it], l[J-1, i, 0, 0, it], J-1, 0,0, it)
    
    interpolate(J-1, it)
    
    for ij in range(J-2, j-1, -1):
        
        it = year(t, j, ij)
        
        if ij >= JR-1:
            ipmax = 1
            ismax = 1
        else:
            ipmax = NP
            ismax = NS
            
        for i in range(NA+1):
            if (ij >= JR-1 and i == 0 and κ[t] <= 1e-10):
                aplus[ij,i,:,:,it] = 0
                c[ij,i,:,:,it] = 0
                l[ij,i,:,:,it] = 0
                VV[ij,i,:,:,it] = valuefunc(0,0,0,ij,0,0,it)
                continue
            
            for p in range(ipmax):
                for s in range(ismax):
                    res = root(foc, x0 = aplus[ij,i,p,s,it], args = (i, ij, p, s, it), tol = 1e-8)
                    if res.x[0] < 0:
                        aplus[ij,i,p,s,it] = 0
                        wage = wn[it]*eff[ij]*θ[p]*η[s]
                        vind = v[ij,i,p,s,it]
                        available = (1+rn[it])*a[i] + pen[ij,it] + vind
                        if ij < JR-1:
                            labcom = min(max(ν-(1-ν)*available/wage,0), 1-1e-10)
                        else:
                            labcom = 0
                        conscom = max((available + wage *labcom)/pp[it],1e-10)
                    else:
                        ap = res.x[0]
                        vind = v[j, i, p, s, t]
                        wage = wn[t]*eff[j]*θ[p]*η[s]
                        available = (1 + rn[t])*a[i] + pen[j,t] + vind
                        if j < JR-1:
                            labcom = min(max(ν + (1-ν)*(ap-available)/wage, 0),1-1e-10)
                        else:
                            labcom = 0
                        conscom = max((available + wage*labcom -ap)/pp[t],1e-10) 
                    
                    aplus[ij,i,p,s,it] = res.x[0]
                    c[ij,i,p,s,it] = conscom
                    l[ij,i,p,s,it] = labcom
                    VV[ij,i,p,s,it] = valuefunc(res.x[0], conscom, labcom, ij, p, s, it)
                
                if ij >= JR - 1:
                    aplus[ij,i,:,:,it] = aplus[ij,i,0,0,t]
                    c[ij,i,:,:,it] = c[ij,i,0,0,it]
                    l[ij,i,:,:,it] = l[ij,i,0,0,it]
                    VV[ij,i,:,:,it] = VV[ij,i,0,0,it]
            
        interpolate(ij,it)
        
    return None
    
    
    
    

In [11]:
def get_distribution(t):
    tm = year(t,1,0)
    
    Φ[:,:,:,:,t] = 0
    
    for p in range(NP):
        Φ[0,0,p,2,t] = distθ[p]
        
    for j in range(1,J):
        for i in range(NA+1):
            for p in range(NP):
                for s in range(NS):
                    
                    ial, iar, φ = LinintGrow(aplus[j-1,i,p,s,tm], al, au, agrow, NA+1)
                    
                    ial = max(min(ial, NA-1),0)
                    iar = max(min(iar, NA)  ,1)
                    φ   = max(min(φ  , 1 )  ,0)
                    
                    for sp in range(NS):
                        Φ[j,ial,p,sp,t] += π[s,sp]*φ    *Φ[j-1,i,p,s,tm]
                        Φ[j,iar,p,sp,t] += π[s,sp]*(1-φ)*Φ[j-1,i,p,s,tm]
    
    
    return None
    

In [12]:
def aggregation(t):
    
    tp = year(t,0,1)
    
    Lold = L[t]
    
    c_coh[:,t] = 0
    l_coh[:,t] = 0
    y_coh[:,t] = 0
    a_coh[:,t] = 0
    VV_coh[:,t] = 0
    FLC[:,t] = 0
    
    m_coh = np.zeros(J)
    
    for j in range(J):
        for i in range(NA+1):
            for p in range(NP):
                for s in range(NS):
                    
                    c_coh[j,t] += c[j,i,p,s,t]*Φ[j,i,p,s,t]
                    l_coh[j,t] += l[j,i,p,s,t]*Φ[j,i,p,s,t]
                    y_coh[j,t] += eff[j]*θ[p]*η[s]*l[j,i,p,s,t]*Φ[j,i,p,s,t]
                    a_coh[j,t] += a[i]*Φ[j,i,p,s,t]
                    
                    if j>=JR-1 and i == 0 and (κ[0]<= 1e-10 or κ[1]<=1e-10):
                        continue
                    if aplus[j,i,p,s,t] < 1e-4:
                        FLC[j,t] += Φ[j,i,p,s,t]
                    VV_coh[j,t] += VV[j,i,p,s,t]*Φ[j,i,p,s,t]
                    m_coh[j] += Φ[j,i,p,s,t]

    
    VV_coh[:,t] = VV_coh[:,t]/m_coh
    FLC[:,t] = FLC[:,t]/m_coh
    
    
    
    C[t] = 0
    L[t] = 0
    H[t] = 0
    A[t] = 0
    workpop = 0
    
    for j in range(J):
        C[t] += c_coh[j,t]*m[j,t]
        L[t] += y_coh[j,t]*m[j,t]
        H[t] += l_coh[j,t]*m[j,t]
        A[t] += a_coh[j,t]*m[j,t]
        
        if j < JR-1:
            workpop += m[j,t]
    
    
    K[t] = damp*(A[t] - B[t] - BA[t]) + (1-damp)*K[t]
    L[t] = damp*L[t] + (1-damp)*Lold
    I[t] = (1+n_p)*K[tp] - (1-δ)*K[t]
    Y[t] = Ω * K[t]**α *L[t]**(1-α)
    
    INC[t] = w[t]*L[t]/workpop
    H[t] = H[t]/workpop
    
    DIFF[t] = Y[t] -C[t]- I[t] -G[t]
    
    return None
                        



In [13]:
def government(t):
    tm = year(t,1,0)
    tp = year(t,0,1)
    
    G[t] = gy*Y[0]
    B[t] = by*Y[0]
    pen[JR-1:, t] = κ[t]*INC[tm]
    PP[t] = 0
    
    for j in range(JR-1,J):
        PP[t] += pen[j,t]*m[j,t]
        
    expand = G[t] + (1+r[t])*B[t] - (1+n_p)*B[tp]
    
    if tax[t] == 1:
        τc[t] = (expand - (τw[t]*w[t]*L[t] + τr[t]*r[t]*A[t]))/C[t]
        pp[t] = 1 + τc[t]
    
    elif tax[t] == 2:
        τw[t] = (expand - τc[t]*C[t])/(w[t]*L[t] + r[t]*A[t])
        τr[t] = τw[t]
    elif tax[t] == 3:
        τw[t] = (expand - (τc[t]*C[t] + τr[t]*r[t]*A[t]))/(w[t]*L[t])
    else:
        τr[t] = (expand - (τc[t]*C[t] + τw[t]*w[t]*L[t]))/(r[t]*A[t])
        
    
    taxrev[0,t] = τc[t]*C[t]
    taxrev[1,t] = τw[t]*w[t]*L[t]
    taxrev[2,t] = τr[t]*r[t]*A[t]
    taxrev[3,t] = sum(taxrev[:3,t])
    
    τp[t] = PP[t]/(w[t]*L[t])
    
    return None









In [19]:
def get_SteadyState():
    
    for ite in range(itermax):
        prices(0)
        solve_household(0,0)
        get_distribution(0)
        aggregation(0)
        government(0)
        
        print(ite, H[0], DIFF[0]/Y[0])
        
        if abs(DIFF[0]/Y[0])*100 < sig:
            break
    return None
        
        

    

In [20]:
get_SteadyState()

0 0.7772269713815071 -0.08283077062210237
1 0.4259693648149431 -1.169293261533033
2 0.7368921628025928 -0.4852337756948271
3 0.8316969268445349 -0.4449854142424349
4 0.878951259237086 -0.4329727879550011
5 0.9194633704627819 -0.409231888546841
6 0.9526263732175976 -0.3859378360355784
7 0.9584659044536217 -0.3932649174148103
8 0.9604964342148804 -0.4023897634642528
9 0.8595408735418069 -0.5120963734885391
10 0.8276965182958421 -0.5649620850470556
11 0.5247480216870126 -0.8707887522395619
12 0.7545663147214869 -0.6655358916172365
13 0.9221231988812167 -0.5360655351157708
14 0.8837398653098054 -0.625694065417291
15 0.8962730720697374 -0.6895438959378807
16 0.8999471468855295 -0.7349045415512576
17 0.909116069942459 -0.7606014186478363
18 0.9155046428951183 -0.7763219835907774
19 0.920073589564797 -0.7857846929743908
20 0.9234362464485643 -0.7913714137114173
21 0.9259415517587711 -0.7944617272834404
22 0.9277052066778109 -0.7962067462136864
23 0.9290926224129376 -0.7969702121498703
24 0.92

In [23]:
print(aplus[:,:,:,:,0])

[[[[ 1.99705665e+05  7.19230247e+03  5.95448949e+05  3.30886937e+07
     1.45528278e+08]
   [ 2.16481408e+07  1.87533456e+06  7.41337078e+07  7.96183227e+07
     2.65510965e+06]]

  [[ 3.74889137e+07  5.96311572e+07  1.74684001e+07  5.98077672e+06
     2.56703232e+06]
   [ 6.58609190e+07  1.07523520e+07  8.38684777e+06  7.38667092e+07
     3.23725039e+06]]

  [[ 4.60838030e+05  4.20928071e+06  7.59975709e+04  1.57882239e+07
     2.32643754e+05]
   [ 3.60736482e+07  1.40598711e+06  5.19387853e+07  6.76891346e+06
     6.24420005e+05]]

  ...

  [[ 6.73132203e+07  2.61595280e+07  1.61975415e+08  1.38692820e+07
     5.23484013e+07]
   [ 1.68006885e+07  6.28985561e+06  2.59047008e+07  1.26353979e+05
     2.60977890e+07]]

  [[ 1.24159001e+05  1.17460387e+08  2.99019806e+07  1.83706677e+08
     5.97243576e+05]
   [ 3.31134122e+05  1.98638409e+05  4.97800116e+05  3.15805330e+07
     4.77719474e+06]]

  [[ 6.25691896e+07  1.28637726e+08  3.34148833e+07  6.09028227e+06
     3.36085656e+07]
   [